In [100]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Machine Learning Effector Classification</h1>
<div id="toc"></div>

## Overview of ML Pipeline



    
    
**1. Load in positive and negative training sets**


    a. Positive: (effectors)
        - Oomycete effectors from Phi-base
        - Bremia WY AVR and other AVR candidates
        
    b. Negative: (non-effectors)
        - secreted orthologs, used as a proxy. 1:1 positive:negative ratio
        - clustered with CD-Hit to get centroids
        
**2. Run various machine learning models, compare results**

    a. Models used: 
        - RandomForestClassifier(n_estimators=200, max_depth=3, random_state=1),
        - LogisticRegression(random_state=1),
        - KNeighborsClassifier(n_neighbors=5),
        - BernoulliNB(),
        - GaussianNB()
            
    b. Compare results:
        - use results with best accuracy percentage in 1:1 ratio

**3. Save model with best accuracy, do visualization**

        - Plot PR curves
        - Confusion matrices
        
**4. Plot traits of effectors vs non-effectors**

        - for all traits, create ribbon plots to see which is most distinctive
            - do statistical tests for all comparisons
            
</font>



<h1 id="tocheading">Preprocessing and Data Wrangling:</h1>

## 1. Import modules needed

In [101]:
## preprocessing and viz
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import plotnine as p9
from Bio import SeqIO

## https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
np.random.seed(1338)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
n_splits = 4

## machine learning
from sklearn import neighbors, preprocessing, metrics, model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
import pickle

import random


## sklearn models
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import FEAT as FEAT


## 2. Load in Effector training set, create Dataframe <a class="anchor" id="second-bullet"></a>

In [102]:
def makeDataFrame(inFile, effector='1'):
    
    fasta_sequences = SeqIO.parse(open(inFile),'fasta')
    
    # extract relevant columns
    myColumns = ['protID', 'species', 'sequence', 'effector']
    df_ = pd.DataFrame(columns=myColumns)

    # create panda series entry for each gene
    for prot in fasta_sequences:
        name, sequence = prot.id, str(prot.seq)
        id_contents = name.split('#')
        entry = pd.Series([id_contents[0], id_contents[4], sequence, effector], index=myColumns)
        df_ = df_.append(entry, ignore_index=True)

    print(df_.groupby('species').count())

    # initialize feature rows
    df_['gravy'] = None
    df_['hydrophobicity'] = None
    df_['exposed'] = None
    df_['disorder'] = None
    df_['bulkiness'] = None
    df_['interface'] = None

    return df_

In [103]:
# parse newly made file of combined sequences for positive training set
input_file = "/Users/munir/Desktop/oomycete-effector-prediction/machine_learning_classification/training_data/positive_training_set_effectors.fasta"

df_ = makeDataFrame(input_file)

/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and w

                                protID  sequence  effector
species                                                   
Bremia_lactucae                     15        15        15
Hyaloperonospora_arabidopsidis      39        39        39
Phytophthora_cactorum                1         1         1
Phytophthora_infestans              13        13        13
Phytophthora_parasitica              2         2         2
Phytophthora_sojae                  21        21        21
Pythium_aphanidermatum               1         1         1


/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/423549372.py:14: FutureWarning: The frame.append method is deprecated and w

In [104]:
df_

,protID,species,sequence,effector,gravy,hydrophobicity,exposed,disorder,bulkiness,interface
0,A0A0M5K865,Phytophthora_sojae,MVKLYCAVVGVAGSAFSVRVDESDTVDDLKDAIKAKKPNDFKDIDA...,1,None,None,None,None,None,None
1,A4L9T6,Phytophthora_sojae,MRLTNTLVVAVAAILLASENAFSAATDADQATVSKLAAAEFDTLVD...,1,None,None,None,None,None,None
2,A4LAA7,Phytophthora_sojae,MRLTNTLVVAVAAILLASENAFSAATDADQATVSKFAAAEFDTLVD...,1,None,None,None,None,None,None
3,A5YTY8,Phytophthora_sojae,MRLAQVVVVIAASFLVATDALSTTNANQAKIIKGTSPGGHSPRLLR...,1,None,None,None,None,None,None
4,A7L812,Phytophthora_sojae,MGLHKGFFVAVALLALLIVAAPADAITDESQPRDATIVDAPLTGRG...,1,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
87,SW11_Catelyn,Bremia_lactucae,MFSMLTILVAFHVARAATSLMAKQDVLPRANVSARNDKVTRMNTTP...,1,None,None,None,None,None,None
88,SW13_Gaia,Bremia_lactucae,MVFPALFLLIAFAFGDAAATSMSNSLQTIPFSGTSVVSRRLNLDPV...,1,None,None,None,None,None,None
89,SW14_Rickon,Bremia_lactucae,MRPKLKRPTSRRCFFLLLFFGFTLLACLVTFSLKTLTFTTTAKALN...,1,None,None,None,None,None,None
90,RW1_Artemis,Bremia_lactucae,MRFVLVIFMTAATICASRETISVEKDGEHDKLLVITSTTRYLKEDY...,1,None,None,None,None,None,None


## 3. Load in non-effector training set

In [105]:
input_file = "/Users/munir/Desktop/oomycete-effector-prediction/machine_learning_classification/training_data/negative_training_set_orthologs.fasta"
#input_file = "/Users/munir/Desktop/bst227/data/negatives/20190512_negatives.fasta"

fasta_sequences = SeqIO.parse(open(input_file),'fasta')

gene_dic = {}

for prot in fasta_sequences:
    gene_dic[str(prot.id)] = str(prot.seq) 

balanced = random.sample(gene_dic.items(), int(92))


for protID, seq in balanced:
    entry = pd.Series([protID, 'mix', seq, '0' ],\
                      index=['protID', 'species', 'sequence', 'effector'])
    df_ = df_.append(entry, ignore_index=True)

/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and w

/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and w

/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/var/folders/tl/vz96dccx64l1pslnvt84v9v00000gq/T/ipykernel_33086/880765793.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [106]:
df_

,protID,species,sequence,effector,gravy,hydrophobicity,exposed,disorder,bulkiness,interface
0,A0A0M5K865,Phytophthora_sojae,MVKLYCAVVGVAGSAFSVRVDESDTVDDLKDAIKAKKPNDFKDIDA...,1,None,None,None,None,None,None
1,A4L9T6,Phytophthora_sojae,MRLTNTLVVAVAAILLASENAFSAATDADQATVSKLAAAEFDTLVD...,1,None,None,None,None,None,None
2,A4LAA7,Phytophthora_sojae,MRLTNTLVVAVAAILLASENAFSAATDADQATVSKFAAAEFDTLVD...,1,None,None,None,None,None,None
3,A5YTY8,Phytophthora_sojae,MRLAQVVVVIAASFLVATDALSTTNANQAKIIKGTSPGGHSPRLLR...,1,None,None,None,None,None,None
4,A7L812,Phytophthora_sojae,MGLHKGFFVAVALLALLIVAAPADAITDESQPRDATIVDAPLTGRG...,1,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
179,H257_08719-t26_1,mix,MYASRGDEHVAARKEWFFRRLLSSDVRQRAASIQKIRVELLEMEPH...,0,None,None,None,None,None,None
180,PUG3_T007423-RA,mix,MPSSLGSRYVPFWVIDDTCKFGALLAFLSKRRMNDAGLDPQLLFFA...,0,None,None,None,None,None,None
181,H310_03456-t26_1,mix,MKDDTAFTSMRRDDTIQSDEYGIHHIQVDTPAIADIEAAMVYAGAL...,0,None,None,None,None,None,None
182,PHYCA_527340T0,mix,MVIVAENLPLPKALHPPPFESSKKIRVLCLHGFRTNKEVMQDQTRG...,0,None,None,None,None,None,None


In [107]:
def get_average_features(df, protID, sequence):
    '''
    Method: Fills feature columns in df with net averages
    Input: 
        - df: dataframe
        - protID: sequence identifier
        - sequence: amino acid string
    '''
    gravy = hydrophobicity = exposed = disorder = bulkiness = interface = 0.0
    
    ## get NET cumulative sums
    #print("calculating for", sequence)
    
    length = min(len(sequence), 500)

    for ind,aa in enumerate(sequence):
        
        if ind == length: 
            break
            
        if aa.upper() in FEAT.INTERFACE_DIC:
            gravy += FEAT.GRAVY_DIC[aa.upper()]
            hydrophobicity += FEAT.HYDRO_DIC[aa.upper()]
            exposed += FEAT.EXPOSED_DIC[aa.upper()]
            disorder += FEAT.DISORDER_DIC[aa.upper()]
            bulkiness += FEAT.BULKY_DIC[aa.upper()]
            #if not ind > 80:
            interface += FEAT.INTERFACE_DIC[aa.upper()]

    
    ## store averages in df_
    df.loc[protID, 'gravy'] = gravy / length
    df.loc[protID, 'hydrophobicity'] = hydrophobicity / length
    df.loc[protID, 'exposed'] = exposed / length
    df.loc[protID, 'disorder'] = disorder / length
    df.loc[protID, 'bulkiness'] = bulkiness / length
    df.loc[protID, 'interface'] = interface / length

In [108]:
## establish feature values for each protein
for protID, entry in df_.iterrows():
    get_average_features(df_, protID, entry['sequence'])

In [109]:
df_

,protID,species,sequence,effector,gravy,hydrophobicity,exposed,disorder,bulkiness,interface
0,A0A0M5K865,Phytophthora_sojae,MVKLYCAVVGVAGSAFSVRVDESDTVDDLKDAIKAKKPNDFKDIDA...,1,-0.2882,0.33544,-0.1732,0.13,15.21142,0.06278
1,A4L9T6,Phytophthora_sojae,MRLTNTLVVAVAAILLASENAFSAATDADQATVSKLAAAEFDTLVD...,1,-0.469421,0.22562,-0.26281,0.132231,15.28686,0.015041
2,A4LAA7,Phytophthora_sojae,MRLTNTLVVAVAAILLASENAFSAATDADQATVSKFAAAEFDTLVD...,1,-0.325397,0.274762,-0.20873,0.142857,15.157778,0.035317
3,A5YTY8,Phytophthora_sojae,MRLAQVVVVIAASFLVATDALSTTNANQAKIIKGTSPGGHSPRLLR...,1,-0.474775,0.232162,-0.244144,0.225225,15.039369,0.01027
4,A7L812,Phytophthora_sojae,MGLHKGFFVAVALLALLIVAAPADAITDESQPRDATIVDAPLTGRG...,1,0.020325,0.398293,-0.113821,0.211382,15.217561,0.036992
...,...,...,...,...,...,...,...,...,...,...
179,H257_08719-t26_1,mix,MYASRGDEHVAARKEWFFRRLLSSDVRQRAASIQKIRVELLEMEPH...,0,-0.3786,0.34738,-0.1102,0.15,14.68122,0.07632
180,PUG3_T007423-RA,mix,MPSSLGSRYVPFWVIDDTCKFGALLAFLSKRRMNDAGLDPQLLFFA...,0,0.36342,0.573872,0.005701,0.004751,15.576532,0.121853
181,H310_03456-t26_1,mix,MKDDTAFTSMRRDDTIQSDEYGIHHIQVDTPAIADIEAAMVYAGAL...,0,0.4872,0.60014,0.0534,-0.054,15.5024,0.15618
182,PHYCA_527340T0,mix,MVIVAENLPLPKALHPPPFESSKKIRVLCLHGFRTNKEVMQDQTRG...,0,-0.302448,0.378322,-0.136364,0.115385,14.971923,0.090909


In [110]:
data = df_.to_numpy()
data = data[:,3:]

In [111]:
X = data[:,1:]
Y = data[:,0]

<h1 id="tocheading">Classification Cross-Validation Trials:</h1>

In [112]:
sens = []
spec = []
acc = []
false_pos = []

def classify_me(features, target, models):
    '''
    takes in predictors and target, performs k cross validation and returns average score
    
    input:
    
    output:
        models_df: data frame that contains the scores of the cross-validation for each model we are testing. 
            We are testing 5 models with k=10 cross validations, so total of 50 scores
        models_df.mean(axis=0): the average of the 10 validations, for each model
    '''
    trained_models = []
    model_evaluations = []
    cv_value = 5 # number of k-fold cross validations you want
    
    sss = StratifiedShuffleSplit(n_splits=cv_value, test_size=0.33)#, random_state=0)

    ## PLOTTING
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import figure
    plt.figure()
    figure(num=None, figsize=(8, 6), dpi=120)
    
    for i, model in enumerate(models):
        model_scores = []
        fitted_models = []
        
        fold = 1
        for train_index, test_index in sss.split(features, target):
        
           
            setTrain = set(train_index)
            setTest = set(test_index)
            
            assert(len(setTrain.intersection(setTest)) == 0)
            
            fitted = model.fit(features[train_index], target[train_index])
            model_scores.append(fitted.score(features[test_index], target[test_index]))
            
            fitted_models.append(fitted)
            
            ## SHOWING AVG SENSITIVITY and SPECIFICITY
            from sklearn.metrics import confusion_matrix
            if names[i] == "Gaussian Naive Bayes":
                
                clf = [CustomThreshold(model, threshold) for threshold in [0.5]]
                CM = confusion_matrix(np.float32(target[test_index]), 
                                      np.float32(clf[0].predict(features[test_index])))
                TN=(CM[0][0])
                FN=(CM[1][0])
                TP=(CM[1][1])
                FP=(CM[0][1])
                
                acc.append( (TN + TP) / (TN + FN + TP + FP) )
                sens.append( TP / (TP + FN) )
                spec.append( TN / (TN + FP) )
                false_pos.append( FP / (FP + TN) )
                                      
                                      

            
            
            ## PLOTTING first FOLD
            if (fold == 1):
                fpr, tpr, thresholds = metrics.roc_curve(np.float32(target[test_index]), model.predict_proba(features[test_index])[:,1])
                # Calculate Area under the curve to display on the plot
                auc = metrics.roc_auc_score(np.float32(target[test_index]),np.float32(model.predict(features[test_index])))
                # Now, plot the computed values
#                 plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (names[i], auc))
            
            #print("TRAIN:", train_index, "TEST:", test_index, "\n")
            #print("****\nTRAIN IS LENGTH: ", len(train_index), "*****\n\n")
            #print("\n\ntarget[test]:", target[test_index], "\n")
            fold = fold + 1
            
        trained_models.append(fitted_models)
        model_evaluations.append(model_scores)
        
        #cur_scores = cross_val_score(model, features_train, target_train, cv=cv_value)
        #model_evaluations.append(cur_scores)
    
    
    # this spits out the results from the k-folds cross validations, and then gives us the averages as well.
    model_names = [model.__class__.__name__ for model in models]
    models_df = pd.DataFrame(model_evaluations).T
    models_df.columns = model_names
    
    ## PLOTTING
#     plt.plot([0, 1], [0, 1],'r--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('1-Specificity(False Positive Rate)', fontsize=15)
#     plt.ylabel('Sensitivity(True Positive Rate)',fontsize=15)
#     plt.legend(loc="lower right")
#     #plt.show()   # Display
    
    return(models_df, models_df.mean(axis=0), trained_models)

models = [
    #LinearSVC(),
    #RandomForestClassifier(n_estimators=400, max_depth=2, random_state=1),
    #RandomForestClassifier(n_estimators=400, max_depth=3, random_state=1),
    RandomForestClassifier(n_estimators=400, max_depth=4, random_state=1), #best :)
    #RandomForestClassifier(n_estimators=600, max_depth=4, random_state=1),
    #RandomForestClassifier(n_estimators=400, max_depth=5, random_state=1),
    LogisticRegression(random_state=1),
    KNeighborsClassifier(n_neighbors=5),
    BernoulliNB(),
    GaussianNB()
]

names = ["Random forest", "Logistic regression", "K-nearest neighbors", 
         "Bernoulli Naive Bayes", "Gaussian Naive Bayes"]

In [113]:
## get features columns
features = (df_.values[:,4:])

## get target values
target = df_.values[:,3]

#features = preprocessing.scale(features)

allResults, avgResults, my_models = classify_me(features, target, models)
print("Performance for each model, at each k-folds: \n\n{}".format(allResults))

print("Average performace for each model:\n\n{}".format(avgResults))

Performance for each model, at each k-folds: 

   RandomForestClassifier  LogisticRegression  KNeighborsClassifier  \
0                0.836066            0.524590              0.639344   
1                0.770492            0.622951              0.721311   
2                0.803279            0.655738              0.737705   
3                0.836066            0.491803              0.754098   
4                0.721311            0.590164              0.655738   

   BernoulliNB  GaussianNB  
0     0.557377    0.737705  
1     0.573770    0.590164  
2     0.590164    0.688525  
3     0.573770    0.639344  
4     0.639344    0.655738  
Average performace for each model:

RandomForestClassifier    0.793443
LogisticRegression        0.577049
KNeighborsClassifier      0.701639
BernoulliNB               0.586885
GaussianNB                0.662295
dtype: float64


<Figure size 432x288 with 0 Axes>

<Figure size 960x720 with 0 Axes>

In [114]:
allResults

,RandomForestClassifier,LogisticRegression,KNeighborsClassifier,BernoulliNB,GaussianNB
0,0.836066,0.524590,0.639344,0.557377,0.737705
1,0.770492,0.622951,0.721311,0.573770,0.590164
2,0.803279,0.655738,0.737705,0.590164,0.688525
3,0.836066,0.491803,0.754098,0.573770,0.639344
4,0.721311,0.590164,0.655738,0.639344,0.655738
